In [1]:
!pip install pandas numpy scikit-learn nltk joblib matplotlib seaborn #installing necessary libaries


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import joblib
import matplotlib.pyplot as plt
import seaborn as sns #importing all the libaries


In [3]:

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [4]:

nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
fake_df = pd.read_csv("Untitled Folder 1/True.csv") #reading csv file



FileNotFoundError: [Errno 2] No such file or directory: 'Untitled Folder 1/True.csv'

In [15]:


os.listdir() #listing files


['.ipynb_checkpoints', 'Fake News Detector.ipynb', 'True.csv']

In [16]:
true_df = pd.read_csv("True.csv") #reading true csv file


In [17]:
fake_df = pd.read_csv("Fake News project.csv") #reading false csv file

In [18]:
true_df.head()
 # viewing 5 first columns details


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [19]:

fake_df.head() #also viewing first five fake news details

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [20]:
true_df['label'] = 1   # REAL
fake_df['label'] = 0   # FAKE


In [21]:
df = pd.concat([true_df, fake_df], axis=0)
df = df.sample(frac=1).reset_index(drop=True)


In [22]:
df.head()

,title,text,subject,date,label
0,Factbox: What might be in Senate's mystery hea...,(Reuters) - U.S. Senate Republicans were expec...,politicsNews,"July 25, 2017",1
1,"BREAKING: CROOKED VA GOVERNOR, Close Hillary F...",How much more criminal activity are American v...,politics,"Oct 23, 2016",0
2,HOLLYWOOD’S SLICK PRO-HILLARY AD Gets A TRUMP ...,A bunch of celebrities set out to hijack our e...,politics,"Sep 23, 2016",0
3,Factbox: Key players in South Africa's ANC lea...,JOHANNESBURG (Reuters) - South Africa s ruling...,worldnews,"December 16, 2017",1
4,Gunman injures bodyguard of Kenya's deputy chi...,NAIROBI (Reuters) - A gunman shot and wounded ...,worldnews,"October 24, 2017",1


In [23]:
df.columns

Index(['title', 'text', 'subject', 'date', 'label'], dtype='object')

In [24]:
df.isna() #checking null values

,title,text,subject,date,label
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
44893,False,False,False,False,False
44894,False,False,False,False,False
44895,False,False,False,False,False
44896,False,False,False,False,False


In [25]:
df.isna().sum() #sum of nan values

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [26]:
df.dropna(inplace=True) #removes the nan values if any

In [27]:
df['label'].value_counts() #checks the number of fake and true news


label
0    23481
1    21417
Name: count, dtype: int64

In [28]:
df['content'] = df['title'] + " " + df['text']


In [29]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['clean_content'] = df['content'].apply(preprocess_text) #preprocessing the text


In [30]:
print("BEFORE:\n", df['content'][0][:300])
print("\nAFTER:\n", df['clean_content'][0][:300]) #analysing after preprocessinf


BEFORE:
 Factbox: What might be in Senate's mystery healthcare bill (Reuters) - U.S. Senate Republicans were expected to vote on Tuesday for opening debate on a healthcare bill without knowing what is in it, but the potential components of the legislation were clear.  Approaches hammered out by Republicans i

AFTER:
 factbox might senate mystery healthcare bill reuters u senate republican expected vote tuesday opening debate healthcare bill without knowing potential component legislation clear approach hammered republican recent month expected mix senate majority leader mitch mcconnell making key choice based mu


In [31]:
X = df['clean_content']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
) #train ,test and split


In [32]:
vectorizer = TfidfVectorizer(
    max_df=0.7,
    ngram_range=(1,2)
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test) #tfidfvectorizer


In [33]:
#for logistic Regression model
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_tfidf, y_train)

lr_pred = lr.predict(X_test_tfidf)
lr_acc = accuracy_score(y_test, lr_pred)


MemoryError: Unable to allocate 36.8 MiB for an array with shape (9643131,) and data type int32

In [34]:
vectorizer = TfidfVectorizer(
    max_features=50000,   # LIMIT FEATURES
    max_df=0.7,
    min_df=5,             # remove rare words
    stop_words='english'
)


In [35]:
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [36]:
print(X_train_tfidf.shape)

(35918, 31169)


In [37]:

#Naive bayes model
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

nb_pred = nb.predict(X_test_tfidf)
nb_acc = accuracy_score(y_test, nb_pred)

print("Naive Bayes Accuracy:", nb_acc)


Naive Bayes Accuracy: 0.9304008908685969


In [38]:
#for logistic regression model

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(
    max_iter=1000,
    solver='saga',
    n_jobs=-1
)

lr.fit(X_train_tfidf, y_train)

lr_pred = lr.predict(X_test_tfidf)
lr_acc = accuracy_score(y_test, lr_pred)

print("Logistic Regression Accuracy:", lr_acc)


Logistic Regression Accuracy: 0.983630289532294


In [39]:
#for support vector machine

from sklearn.svm import LinearSVC

svm = LinearSVC()
svm.fit(X_train_tfidf, y_train)

svm_pred = svm.predict(X_test_tfidf)
svm_acc = accuracy_score(y_test, svm_pred)

print("SVM Accuracy:", svm_acc)


SVM Accuracy: 0.9927616926503341


In [40]:
#For random forest classifier

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train_tfidf, y_train)

rf_pred = rf.predict(X_test_tfidf)
rf_acc = accuracy_score(y_test, rf_pred)

print("Random Forest Accuracy:", rf_acc)


Random Forest Accuracy: 0.9853006681514477


In [41]:
import pandas as pd

Results = pd.DataFrame({
    "Model": [
        "Naive Bayes",
        "Logistic Regression",
        "Support Vector Machine",
        "Random Forest"
    ],
    "Accuracy": [
        nb_acc,
        lr_acc,
        svm_acc,
        rf_acc
    ]
})

Results


,Model,Accuracy
0,Naive Bayes,0.930401
1,Logistic Regression,0.983630
2,Support Vector Machine,0.992762
3,Random Forest,0.985301


In [43]:
import joblib

joblib.dump(best_model, "fake_news_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']

In [45]:
def predict_news(news):
    cleaned = preprocess_text(news)
    vectorized = vectorizer.transform([cleaned])
    pred = best_model.predict(vectorized)
    return "REAL NEWS" if pred[0] == 1 else "FAKE NEWS"

test_news = "Simona is the best"
print(predict_news(test_news))


FAKE NEWS
